<figure>
    <center>
    <img src="../../figures/kinki2.png"  width="400" height="370" align="center"/>
    </center>
</figure>

# <span style="color:blue"><center>Hackathon: Shinkansen Travel Experience</center></span>

# <span style="color:red"><center>Process studied</center></span>

## <span style="color:green"><center> Ensembling learning </center></span>

## <span style="color:blue">Authors</span>

1. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com
1. Juan Felipe Contreras Alcívar, j.contrerasa94@gmail.com

## <span style="color:blue">Libraries</span>

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


import catboost
import xgboost

import warnings
warnings.filterwarnings('ignore')

## <span style="color:blue"> Importing Data </span>

In [2]:
train_df = pd.read_csv('../../input/train/imputed_no_response_df.csv', sep=';', index_col=0)
test_df = pd.read_csv('../../input/test/imputed_no_response_df.csv', sep=';', index_col=0)

train_df

,Overall_Experience,Age,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,Gender,Customer_Type,Type_Travel,Travel_Class,Seat_Comfort,...,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding
ID,,,,,,,,,,,,,,,,,,,,,
98800001,0,52.0,272,0.0,5.0,Female,Loyal Customer,No_response,Business,Needs Improvement,...,Good,Needs Improvement,Acceptable,Needs Improvement,Needs Improvement,Acceptable,Needs Improvement,Good,Needs Improvement,Poor
98800002,0,48.0,2200,9.0,0.0,Male,Loyal Customer,Personal Travel,Eco,Poor,...,Good,Poor,Good,Good,Excellent,Needs Improvement,Poor,Needs Improvement,Good,Good
98800003,1,43.0,1061,77.0,119.0,Female,Loyal Customer,Business Travel,Business,Needs Improvement,...,Needs Improvement,Good,Excellent,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Excellent
98800004,0,44.0,780,13.0,18.0,Female,Loyal Customer,Business Travel,Business,Acceptable,...,Acceptable,Needs Improvement,Acceptable,Acceptable,Acceptable,Acceptable,Acceptable,Good,Acceptable,Acceptable
98800005,1,50.0,1981,0.0,0.0,Female,Loyal Customer,Business Travel,Business,Acceptable,...,Needs Improvement,Good,Excellent,Good,Good,Good,Good,Good,Good,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98894375,0,32.0,1357,83.0,125.0,Male,Loyal Customer,Business Travel,Business,Poor,...,Poor,Poor,Poor,Poor,Good,Good,Good,Needs Improvement,Good,Poor
98894376,1,44.0,592,5.0,11.0,Male,Loyal Customer,Business Travel,Business,Good,...,Needs Improvement,Excellent,Excellent,Acceptable,Acceptable,Acceptable,Acceptable,Good,Acceptable,Good
98894377,1,63.0,2794,0.0,0.0,Male,No_response,Business Travel,Business,Needs Improvement,...,Good,Excellent,Good,Good,Good,Good,Good,Acceptable,Good,Acceptable


## <span style="color:blue"> Feature engineer </span>

In [3]:
train_df['train_test'] = 'train'
test_df['train_test'] = 'test'

df = pd.concat([train_df, test_df])

In [4]:
df

,Overall_Experience,Age,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,Gender,Customer_Type,Type_Travel,Travel_Class,Seat_Comfort,...,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,train_test
ID,,,,,,,,,,,,,,,,,,,,,
98800001,0.0,52.0,272,0.0,5.0,Female,Loyal Customer,No_response,Business,Needs Improvement,...,Needs Improvement,Acceptable,Needs Improvement,Needs Improvement,Acceptable,Needs Improvement,Good,Needs Improvement,Poor,train
98800002,0.0,48.0,2200,9.0,0.0,Male,Loyal Customer,Personal Travel,Eco,Poor,...,Poor,Good,Good,Excellent,Needs Improvement,Poor,Needs Improvement,Good,Good,train
98800003,1.0,43.0,1061,77.0,119.0,Female,Loyal Customer,Business Travel,Business,Needs Improvement,...,Good,Excellent,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Excellent,train
98800004,0.0,44.0,780,13.0,18.0,Female,Loyal Customer,Business Travel,Business,Acceptable,...,Needs Improvement,Acceptable,Acceptable,Acceptable,Acceptable,Acceptable,Good,Acceptable,Acceptable,train
98800005,1.0,50.0,1981,0.0,0.0,Female,Loyal Customer,Business Travel,Business,Acceptable,...,Good,Excellent,Good,Good,Good,Good,Good,Good,Good,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935598,NaN,8.0,1334,0.0,0.0,Male,Loyal Customer,Personal Travel,Eco,Needs Improvement,...,Needs Improvement,Acceptable,Acceptable,Good,Excellent,Good,Acceptable,Good,Acceptable,test
99935599,NaN,53.0,1772,0.0,0.0,Female,Loyal Customer,Business Travel,Business,Needs Improvement,...,Excellent,Excellent,Good,Good,Good,Good,Acceptable,Good,Good,test
99935600,NaN,22.0,1180,0.0,0.0,Male,Disloyal Customer,Business Travel,Eco,Good,...,Good,Poor,Needs Improvement,Poor,Acceptable,Poor,Poor,Excellent,Needs Improvement,test


In [5]:
numerical_columns = ['Age', 'Travel_Distance', 'Departure_Delay_in_Mins', 'Arrival_Delay_in_Mins']

In [6]:
df = pd.get_dummies(df, columns = [x for x in df.columns if x not in ['train_test', 'Overall_Experience'] + numerical_columns])
df

,Overall_Experience,Age,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,train_test,Gender_Female,Gender_Male,Gender_No_response,Customer_Type_Disloyal Customer,...,Cleanliness_Needs Improvement,Cleanliness_No_response,Cleanliness_Poor,Online_Boarding_Acceptable,Online_Boarding_Excellent,Online_Boarding_Extremely Poor,Online_Boarding_Good,Online_Boarding_Needs Improvement,Online_Boarding_No_response,Online_Boarding_Poor
ID,,,,,,,,,,,,,,,,,,,,,
98800001,0.0,52.0,272,0.0,5.0,train,True,False,False,False,...,True,False,False,False,False,False,False,False,False,True
98800002,0.0,48.0,2200,9.0,0.0,train,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
98800003,1.0,43.0,1061,77.0,119.0,train,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
98800004,0.0,44.0,780,13.0,18.0,train,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
98800005,1.0,50.0,1981,0.0,0.0,train,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935598,NaN,8.0,1334,0.0,0.0,test,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
99935599,NaN,53.0,1772,0.0,0.0,test,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
99935600,NaN,22.0,1180,0.0,0.0,test,False,True,False,True,...,False,False,False,False,False,False,False,True,False,False


In [7]:
scaler = StandardScaler()

In [8]:
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
df

,Overall_Experience,Age,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,train_test,Gender_Female,Gender_Male,Gender_No_response,Customer_Type_Disloyal Customer,...,Cleanliness_Needs Improvement,Cleanliness_No_response,Cleanliness_Poor,Online_Boarding_Acceptable,Online_Boarding_Excellent,Online_Boarding_Extremely Poor,Online_Boarding_Good,Online_Boarding_Needs Improvement,Online_Boarding_No_response,Online_Boarding_Poor
ID,,,,,,,,,,,,,,,,,,,,,
98800001,0.0,0.831555,-1.664281,-0.386526,-0.262755,train,True,False,False,False,...,True,False,False,False,False,False,False,False,False,True
98800002,0.0,0.567000,0.213103,-0.150055,-0.392986,train,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
98800003,1.0,0.236305,-0.895995,1.636619,2.706501,train,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
98800004,0.0,0.302444,-1.169618,-0.044957,0.075844,train,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
98800005,1.0,0.699278,-0.000148,-0.386526,-0.392986,train,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935598,NaN,-2.078556,-0.630162,-0.386526,-0.392986,test,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
99935599,NaN,0.897694,-0.203661,-0.386526,-0.392986,test,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
99935600,NaN,-1.152612,-0.780119,-0.386526,-0.392986,test,False,True,False,True,...,False,False,False,False,False,False,False,True,False,False


## <span style="color:blue"> Train/test split </span>

In [9]:
train_df = df[df['train_test']=='train']
test_df = df[df['train_test']=='test']

In [10]:
# Define features and target variable
X_train = train_df.drop(columns=['Overall_Experience', 'train_test'])
y_train = train_df['Overall_Experience']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, random_state=42, shuffle=True, test_size=0.2)

X_test = test_df.drop(['train_test', 'Overall_Experience'], axis=1)

In [11]:
y_train.value_counts()

Overall_Experience
1.0    41274
0.0    34229
Name: count, dtype: int64

In [12]:
X_train.shape

(75503, 114)

In [13]:
X_val.shape

(18876, 114)

## <span style="color:blue"> Models </span>

### <span style="color:#4CC9F0"> Random Forest </span>

In [14]:
rf = RandomForestClassifier(
    min_samples_leaf = 10, 
    min_samples_split = 10, 
    n_jobs=-1,
    n_estimators = 1000
)

In [15]:
rf.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=10, min_samples_split=10,
                       n_estimators=1000, n_jobs=-1)

In [16]:
prob_cut = 0.5

In [17]:
# Evaluate the model
y_pred_prob = rf.predict_proba(X_train)
prob_train = y_pred_prob[:, 1]
y_pred_train =  np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_train, y_pred_train)
f1_nn = f1_score(y_train, y_pred_train, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))

Ensemble - Accuracy: 0.9534190694409493, F1 Score: 0.953430512441263
Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.95      0.95     34229
         1.0       0.96      0.95      0.96     41274

    accuracy                           0.95     75503
   macro avg       0.95      0.95      0.95     75503
weighted avg       0.95      0.95      0.95     75503

Confusion Matrix:
 [[32572  1657]
 [ 1860 39414]]


In [18]:
prob_cut = 0.5

In [19]:
# Evaluate the model
y_pred_prob = rf.predict_proba(X_val)
prob_val = y_pred_prob[:, 1]
y_pred_val =  np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_val, y_pred_val)
f1_nn = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Ensemble - Accuracy: 0.9393939393939394, F1 Score: 0.9394159101998145
Classification Report:
               precision    recall  f1-score   support

         0.0       0.93      0.94      0.93      8557
         1.0       0.95      0.94      0.94     10319

    accuracy                           0.94     18876
   macro avg       0.94      0.94      0.94     18876
weighted avg       0.94      0.94      0.94     18876

Confusion Matrix:
 [[8023  534]
 [ 610 9709]]


### <span style="color:#4CC9F0"> Gradient boosting </span>

In [20]:
gb = GradientBoostingClassifier(
    learning_rate = 0.2, 
    max_depth=10, 
    n_estimators=500,
    random_state=42
)

In [21]:
gb.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.2, max_depth=10, n_estimators=500,
                           random_state=42)

In [22]:
# Evaluate the model
y_pred_prob = gb.predict(X_train)
y_pred_train =  np.round(y_pred_prob)

accuracy_nn = accuracy_score(y_train, y_pred_train)
f1_nn = f1_score(y_train, y_pred_train, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))

Ensemble - Accuracy: 1.0, F1 Score: 1.0
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     34229
         1.0       1.00      1.00      1.00     41274

    accuracy                           1.00     75503
   macro avg       1.00      1.00      1.00     75503
weighted avg       1.00      1.00      1.00     75503

Confusion Matrix:
 [[34229     0]
 [    0 41274]]


In [23]:
# Evaluate the model
y_pred_prob = gb.predict(X_val)
y_pred_val =  np.round(y_pred_prob)

accuracy_nn = accuracy_score(y_val, y_pred_val)
f1_nn = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Ensemble - Accuracy: 0.9580949353676627, F1 Score: 0.9581274969224703
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.96      0.95      8557
         1.0       0.97      0.95      0.96     10319

    accuracy                           0.96     18876
   macro avg       0.96      0.96      0.96     18876
weighted avg       0.96      0.96      0.96     18876

Confusion Matrix:
 [[8248  309]
 [ 482 9837]]


### <span style="color:#4CC9F0"> Catboost </span>

In [24]:
cb = catboost.CatBoostClassifier(
    depth = 8, 
    verbose=0, 
    iterations = 1000, 
    l2_leaf_reg = 7, 
    learning_rate = 0.1
    )

In [25]:
cb.fit(X_train, y_train)

In [26]:
prob_cut = 0.5

In [27]:
# Evaluate the model
y_pred_prob = cb.predict_proba(X_train)
prob_train = y_pred_prob[:, 1]
y_pred_train =  np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_train, y_pred_train)
f1_nn = f1_score(y_train, y_pred_train, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))

Ensemble - Accuracy: 0.9941194389626902, F1 Score: 0.9941204726952867
Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     34229
         1.0       1.00      0.99      0.99     41274

    accuracy                           0.99     75503
   macro avg       0.99      0.99      0.99     75503
weighted avg       0.99      0.99      0.99     75503

Confusion Matrix:
 [[34079   150]
 [  294 40980]]


In [28]:
prob_cut = 0.5

In [29]:
# Evaluate the model
y_pred_prob = cb.predict_proba(X_val)
prob_val = y_pred_prob[:, 1]
y_pred_val =  np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_val, y_pred_val)
f1_nn = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Ensemble - Accuracy: 0.9577240940877304, F1 Score: 0.9577550711054104
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.96      0.95      8557
         1.0       0.97      0.95      0.96     10319

    accuracy                           0.96     18876
   macro avg       0.96      0.96      0.96     18876
weighted avg       0.96      0.96      0.96     18876

Confusion Matrix:
 [[8239  318]
 [ 480 9839]]


### <span style="color:#4CC9F0"> XGBoost </span>

In [30]:
xgb = xgboost.XGBClassifier(
    n_estimators=300, 
    max_depth=10,
    max_leaves=64,
    eta=0.1,
    reg_lambda=0,
    tree_method='hist',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

In [31]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=64, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=None, num_parallel_tree=None, ...)

In [32]:
prob_cut = 0.5

In [33]:
# Evaluate the model
y_pred_prob = xgb.predict_proba(X_train)
prob_train = y_pred_prob[:, 1]
y_pred_train =  np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_train, y_pred_train)
f1_nn = f1_score(y_train, y_pred_train, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))

Ensemble - Accuracy: 0.9818020476007576, F1 Score: 0.9818093827732003
Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     34229
         1.0       0.99      0.98      0.98     41274

    accuracy                           0.98     75503
   macro avg       0.98      0.98      0.98     75503
weighted avg       0.98      0.98      0.98     75503

Confusion Matrix:
 [[33712   517]
 [  857 40417]]


In [34]:
prob_cut = 0.5

In [35]:
# Evaluate the model
y_pred_prob = xgb.predict_proba(X_val)
prob_val = y_pred_prob[:, 1]
y_pred_val =  np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_val, y_pred_val)
f1_nn = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Ensemble - Accuracy: 0.9553930917567282, F1 Score: 0.9554261383410158
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.96      0.95      8557
         1.0       0.97      0.95      0.96     10319

    accuracy                           0.96     18876
   macro avg       0.95      0.96      0.96     18876
weighted avg       0.96      0.96      0.96     18876

Confusion Matrix:
 [[8218  339]
 [ 503 9816]]


### <span style="color:#4CC9F0"> Voting classifier </span>

In [36]:
estimators = [('gb', GradientBoostingClassifier(learning_rate = 0.1, 
                                                max_depth=5, 
                                                n_estimators=1000,
                                                random_state=42, 
                                                subsample=0.7)), 
              
              ('rf', RandomForestClassifier(min_samples_leaf = 10, 
                                            min_samples_split = 10, 
                                            n_jobs=-1,
                                            n_estimators = 1000)),
              
              ('cb', catboost.CatBoostClassifier(depth = 8, 
                                                 verbose=0, 
                                                 iterations = 1000, 
                                                 l2_leaf_reg = 7, 
                                                 learning_rate = 0.1,
                                                 scale_pos_weight=(sum(y_train == 0) / sum(y_train == 1)))), 
              
              ('xgb', xgboost.XGBClassifier(n_estimators=300,
                                            max_depth=10,
                                            max_leaves=64,
                                            eta=0.1,
                                            reg_lambda=0,
                                            tree_method='hist',
                                            eval_metric='logloss',
                                            use_label_encoder=False,
                                            random_state=42, 
                                            scale_pos_weight=(sum(y_train == 0) / sum(y_train == 1))))
             ]
              
#              ('ab', AdaBoostClassifier(learning_rate=1, n_estimators=200, random_state=42))]

In [37]:
voting_clf = VotingClassifier(
    estimators = estimators,
    voting='soft'
)

In [38]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(max_depth=5,
                                                         n_estimators=1000,
                                                         random_state=42,
                                                         subsample=0.7)),
                             ('rf',
                              RandomForestClassifier(min_samples_leaf=10,
                                                     min_samples_split=10,
                                                     n_estimators=1000,
                                                     n_jobs=-1)),
                             ('cb',
                              <catboost.core.CatBoostClassifier object at 0x15e921480>),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,...
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=10,
                                            max_leaves=64,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=300, n_jobs=None,
                                            num_parallel_tree=None, ...))],
                 voting='soft')

In [39]:
prob_cut = 0.5

In [40]:
# Evaluate the model
y_pred_prob = voting_clf.predict_proba(X_train)
prob_train = y_pred_prob[:, 1]
y_pred_train =  np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_train, y_pred_train)
f1_nn = f1_score(y_train, y_pred_train, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))

Ensemble - Accuracy: 0.988013721309087, F1 Score: 0.9880193932778369
Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.99      0.99     34229
         1.0       0.99      0.98      0.99     41274

    accuracy                           0.99     75503
   macro avg       0.99      0.99      0.99     75503
weighted avg       0.99      0.99      0.99     75503

Confusion Matrix:
 [[33978   251]
 [  654 40620]]


In [41]:
# Evaluate the model
y_pred_prob = voting_clf.predict_proba(X_val)
prob_val = y_pred_prob[:, 1]
y_pred_val = np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_val, y_pred_val)
f1_nn = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Ensemble - Accuracy: 0.9572472981563891, F1 Score: 0.957286509443123
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.97      0.95      8557
         1.0       0.97      0.95      0.96     10319

    accuracy                           0.96     18876
   macro avg       0.96      0.96      0.96     18876
weighted avg       0.96      0.96      0.96     18876

Confusion Matrix:
 [[8258  299]
 [ 508 9811]]


In [42]:
prob_cut = 0.5

In [43]:
# Evaluate the model
y_pred_prob = voting_clf.predict_proba(X_train)
prob_train = y_pred_prob[:, 1]
y_pred_train =  np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_train, y_pred_train)
f1_nn = f1_score(y_train, y_pred_train, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))

Ensemble - Accuracy: 0.988013721309087, F1 Score: 0.9880193932778369
Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.99      0.99     34229
         1.0       0.99      0.98      0.99     41274

    accuracy                           0.99     75503
   macro avg       0.99      0.99      0.99     75503
weighted avg       0.99      0.99      0.99     75503

Confusion Matrix:
 [[33978   251]
 [  654 40620]]


In [44]:
# Evaluate the model
y_pred_prob = voting_clf.predict_proba(X_val)
prob_val = y_pred_prob[:, 1]
y_pred_val = np.int64(y_pred_prob[:, 1] > prob_cut)

accuracy_nn = accuracy_score(y_val, y_pred_val)
f1_nn = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Ensemble - Accuracy: 0.9572472981563891, F1 Score: 0.957286509443123
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.97      0.95      8557
         1.0       0.97      0.95      0.96     10319

    accuracy                           0.96     18876
   macro avg       0.96      0.96      0.96     18876
weighted avg       0.96      0.96      0.96     18876

Confusion Matrix:
 [[8258  299]
 [ 508 9811]]


In [45]:
prob_cut= 0.5

In [46]:
for clf in voting_clf.estimators_:
    
    y_pred_prob = clf.predict_proba(X_val)    
    prob_val = y_pred_prob[:, 1]
    y_pred_val = np.int64(y_pred_prob[:, 1] > prob_cut)

    print(accuracy_score(y_val, y_pred_val))

0.954227590591227
0.939711803348167
0.9571943208306845
0.9558169103623649


### <span style="color:#4CC9F0"> Bagging Classifiers </span>

#### <span style="color:#4CC9F0"> Catboost </span>

In [47]:
cb = catboost.CatBoostClassifier(
    depth = 8, 
    verbose=0, 
    iterations = 1000, 
    l2_leaf_reg = 7, 
    learning_rate = 0.1
)

In [48]:
bc = BaggingClassifier(
    cb, n_estimators=500,
    n_jobs=-1, max_samples=0.5, bootstrap=False
)

In [49]:
bc.fit(X_train, y_train)

BaggingClassifier(bootstrap=False,
                  estimator=<catboost.core.CatBoostClassifier object at 0x15e9ead40>,
                  max_samples=0.5, n_estimators=500, n_jobs=-1)

In [50]:
# Evaluate the model
y_pred_prob = bc.predict(X_train)
y_pred_train =  np.round(y_pred_prob)

accuracy_nn = accuracy_score(y_train, y_pred_train)
f1_nn = f1_score(y_train, y_pred_train, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))

Ensemble - Accuracy: 0.9881726553911765, F1 Score: 0.9881776217305943
Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.99      0.99     34229
         1.0       0.99      0.98      0.99     41274

    accuracy                           0.99     75503
   macro avg       0.99      0.99      0.99     75503
weighted avg       0.99      0.99      0.99     75503

Confusion Matrix:
 [[33960   269]
 [  624 40650]]


In [51]:
# Evaluate the model
y_pred_prob = bc.predict(X_val)
y_pred_val =  np.round(y_pred_prob)

accuracy_nn = accuracy_score(y_val, y_pred_val)
f1_nn = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Ensemble - Accuracy: 0.957777071413435, F1 Score: 0.9578085226063158
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.96      0.95      8557
         1.0       0.97      0.95      0.96     10319

    accuracy                           0.96     18876
   macro avg       0.96      0.96      0.96     18876
weighted avg       0.96      0.96      0.96     18876

Confusion Matrix:
 [[8241  316]
 [ 481 9838]]


#### <span style="color:#4CC9F0"> XGBoost </span>

In [52]:
xgb = xgboost.XGBClassifier(
    n_estimators=300,
    max_depth=10,
    max_leaves=64,
    eta=0.1,
    reg_lambda=0,
    tree_method='hist',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

In [53]:
bc = BaggingClassifier(
    xgb, 
    n_estimators=500,
    n_jobs=-1, 
    max_samples=0.5, 
    bootstrap=False
)

In [54]:
bc.fit(X_train, y_train)

/Users/juanfelipe/miniforge3/envs/kinki/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:40:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/juanfelipe/miniforge3/envs/kinki/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:40:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/juanfelipe/miniforge3/envs/kinki/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:40:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/juanfelipe/miniforge3/envs/kinki/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:40:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_e

BaggingClassifier(bootstrap=False,
                  estimator=XGBClassifier(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False, eta=0.1,
                                          eval_metric='logloss',
                                          feature_types=None, gamma=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=10,
                                          max_leaves=64, min_child_weight=None,
                                          missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=300,
                                          n_jobs=None, num_parallel_tree=None, ...),
                  max_samples=0.5, n_estimators=500, n_jobs=-1)

In [55]:
# Evaluate the model
y_pred_prob = bc.predict(X_train)
y_pred_train =  np.round(y_pred_prob)

accuracy_nn = accuracy_score(y_train, y_pred_train)
f1_nn = f1_score(y_train, y_pred_train, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_train, y_pred_train))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))

Ensemble - Accuracy: 0.9792061242599632, F1 Score: 0.9792172578881834
Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.98      0.98     34229
         1.0       0.99      0.98      0.98     41274

    accuracy                           0.98     75503
   macro avg       0.98      0.98      0.98     75503
weighted avg       0.98      0.98      0.98     75503

Confusion Matrix:
 [[33674   555]
 [ 1015 40259]]


In [56]:
# Evaluate the model
y_pred_prob = bc.predict(X_val)
y_pred_val =  np.round(y_pred_prob)

accuracy_nn = accuracy_score(y_val, y_pred_val)
f1_nn = f1_score(y_val, y_pred_val, average='weighted')
print(f"Ensemble - Accuracy: {accuracy_nn}, F1 Score: {f1_nn}")
print("Classification Report:\n", classification_report(y_val, y_pred_val))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))

Ensemble - Accuracy: 0.9552871371053189, F1 Score: 0.9553245314076718
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.96      0.95      8557
         1.0       0.97      0.95      0.96     10319

    accuracy                           0.96     18876
   macro avg       0.95      0.96      0.95     18876
weighted avg       0.96      0.96      0.96     18876

Confusion Matrix:
 [[8229  328]
 [ 516 9803]]
